In [4]:
from osgeo import gdal
import numpy as np
import pandas as pd
import richdem as rd
import os
import glob

Constantes

In [42]:
DATA_PATH = './results/data_all_10px_12000.csv'
DATA_INSIVUMEH_PATH = './results/data_insivumeh_all_10px_12000.csv'
RES_DATA = './results/data_train.csv'

Graficar la data

In [35]:
class DefaultMetaObj:
    geotransform = [0,1,0,0,0,-1]

class DataGenerator:
    def __init__(self, data_directory_1, data_directory_2):
        self.data1 = pd.read_csv(data_directory_1, index_col=0)
        print(len(self.data1))
        self.data1.drop_duplicates(keep='first', subset='landslide_id', inplace=True)
        print(len(self.data1))
        self.data2 = pd.read_csv(data_directory_2, index_col=0)
        print(len(self.data2))
        self.data2.drop_duplicates(keep='first', subset='landslide_id', inplace=True)
        print(len(self.data2))

    def getData(self):
        output = pd.merge(self.data1, self.data2, how='inner', on='landslide_id')
        return output

In [36]:
dg = DataGenerator(DATA_PATH, DATA_INSIVUMEH_PATH)
data = dg.getData()

10808
9742
10190
10190


In [37]:
len(data)

9742

In [38]:
insivumeh_columns = [
    'rock_geo','rock_period','fis_nomr','rock_type',
    'soil_rec_symbol','soil_rec_mat_origin','soil_rec_sew_class',
    'soil_rec_tex_sup','soil_rec_c_tex_sup','soil_rec_tex_sub',
    'soil_rec_c__tex_sub','soil_rec_risk_eros','soil_tax','soil_use'
]

for col in insivumeh_columns:
    if '-999' in data[col].value_counts():
        data.loc[data[col] == '-999', col] = pd.NA

In [39]:
insivumeh_columns2 = [
    'soil_rec_mat_origin','soil_rec_sew_class',
    'soil_rec_tex_sup','soil_rec_c_tex_sup','soil_rec_tex_sub',
    'soil_rec_c__tex_sub','soil_rec_risk_eros','soil_tax','soil_use'
]
for col in insivumeh_columns:
        data.loc[data['soil_rec_symbol'] == 'AGUA', col] = 'AGUA'

In [40]:
na_data = data.isnull().sum()
na_data[na_data > 0]

rock_geo                  6
rock_period               6
fis_nomr                  6
rock_type                20
soil_rec_symbol           6
soil_rec_mat_origin     662
soil_rec_sew_class      720
soil_rec_tex_sup        731
soil_rec_c_tex_sup      731
soil_rec_tex_sub        818
soil_rec_c__tex_sub    1212
soil_rec_risk_eros      759
soil_tax                  4
soil_use                  6
dtype: int64

In [62]:
data.dropna(subset=insivumeh_columns, inplace=True)

In [63]:
na_data = data.isnull().sum()
na_data[na_data > 0]

Series([], dtype: int64)

In [41]:
len(data)

9742

In [65]:
data['landslide'].value_counts()

0    6409
1    3033
Name: landslide, dtype: int64

In [13]:
data2 = data
data2['event_date'] = pd.to_datetime(data2['event_date'])

In [14]:
sum = 0
for year in range(2008,2023):
    count = data2[data2.event_date.dt.year==year].count()
    print(year,count['landslide'])
    if year >= 2015:
        sum += count['landslide']
print(sum)

2008 168
2009 16
2010 653
2011 174
2012 63
2013 58
2014 229
2015 82
2016 373
2017 545
2018 426
2019 500
2020 1242
2021 555
2022 147
3870


In [15]:
data2[data2.event_date.dt.year > 2015]['landslide'].value_counts()

0    2213
1    1575
Name: landslide, dtype: int64

In [43]:
data.to_csv(RES_DATA)  